# Atom feed of highly tweeted new preprints by subject area

Preprints published in the past seven days tweeted at least five times as atom feed, grouped by subject area.

In [1]:
import sys
sys.path.append('../')

import ast
import pandas as pd
from datetime import datetime, date
from feedgen.feed import FeedGenerator

In [2]:
subject_areas = ['covid', 'microbiology', 'neuroscience', 'molecular-biology', 'bioinformatics', 'genomics', 'genetics', 'developmental-biology', 'plant-biology', 'evolutionary-biology', 'ecology', 'immunology', 'cancer-biology', 'cell-biology', 'biophysics']
preprints = pd.read_csv('../preprint_tweets/1101/preprint_tweets_' + str(date.today().strftime('%Y-%m-%d')) + '.csv', index_col=0)

# filter by section, and group all COVID-related preprints together
subsets = {}
subsets['covid'] = preprints[preprints['covid'] == True]
subsets['microbiology'] = preprints[(preprints['subject-area'] == 'Microbiology') & (preprints['covid'] == False)]
subsets['neuroscience'] = preprints[(preprints['subject-area'] == 'Neuroscience') & (preprints['covid'] == False)]
subsets['molecular-biology'] = preprints[(preprints['subject-area'] == 'Molecular Biology') & (preprints['covid'] == False)]
subsets['bioinformatics'] = preprints[(preprints['subject-area'] == 'Bioinformatics') & (preprints['covid'] == False)]
subsets['genomics'] = preprints[(preprints['subject-area'] == 'Genomics') & (preprints['covid'] == False)]
subsets['genetics'] = preprints[(preprints['subject-area'] == 'Genetics') & (preprints['covid'] == False)]
subsets['developmental-biology'] = preprints[(preprints['subject-area'] == 'Developmental Biology') & (preprints['covid'] == False)]
subsets['plant-biology'] = preprints[(preprints['subject-area'] == 'Plant Biology') & (preprints['covid'] == False)]
subsets['evolutionary-biology'] = preprints[(preprints['subject-area'] == 'Evolutionary Biology') & (preprints['covid'] == False)]
subsets['ecology'] = preprints[(preprints['subject-area'] == 'Ecology') & (preprints['covid'] == False)]
subsets['immunology'] = preprints[(preprints['subject-area'] == 'Immunology') & (preprints['covid'] == False)]
subsets['cancer-biology'] = preprints[(preprints['subject-area'] == 'Cancer Biology') & (preprints['covid'] == False)]
subsets['cell-biology'] = preprints[(preprints['subject-area'] == 'Cell Biology') & (preprints['covid'] == False)]
subsets['biophysics'] = preprints[(preprints['subject-area'] == 'Biophysics') & (preprints['covid'] == False)]
subsets['cell-biology'].head(50)

,doi,tweets,archive,subject-area,covid,title,authors,abstract,posted
2,10.1101/2021.05.17.444558,198,bioRxiv,Cell Biology,False,Polarized Dishevelled dissolution and condensa...,"[{'name': 'S. Zachary Swartz'}, {'name': 'Tzer...",<p>The organismal body axes that are formed du...,2021-05-18
36,10.1101/2021.05.20.445000,19,bioRxiv,Cell Biology,False,Functional dissection of human mitotic protein...,"[{'name': 'Jacob A. Herman'}, {'name': 'Lucas ...",<p>Kinetochores are large protein assemblies t...,2021-05-20
41,10.1101/2021.05.20.444797,18,bioRxiv,Cell Biology,False,Systematic Tuning of Rhodamine Spirocyclizatio...,"[{'name': 'Nicolas Lardon'}, {'name': 'Lu Wang...",<p>Rhodamines are the most important class of ...,2021-05-20
54,10.1101/2021.05.20.444917,13,bioRxiv,Cell Biology,False,V-ATPase is a universal regulator of LC3 assoc...,"[{'name': 'Kirsty M. Hooper'}, {'name': 'Elise...",<p>Non-canonical autophagy is a key cellular p...,2021-05-20
81,10.1101/2021.05.18.444637,7,bioRxiv,Cell Biology,False,A Drosophila Toolkit for Imaging of HA-tagged ...,"[{'name': 'Tadayoshi Murakawa'}, {'name': 'Tsu...",<p>For <italic>in vivo</italic> functional ana...,2021-05-18


In [3]:

for subject_area in subject_areas:
    fg = FeedGenerator()
    fg.id('https://front-matter.io/')
    if subject_area == 'covid':
        fg.title(f'Popular BioRxiv/medRxiv COVID preprints posted in the last week')
    else:
        fg.title(f'Popular BioRxiv {subject_area.replace("_", " ")} preprints posted in the last week')
    fg.author( {'name':'Martin Fenner','email':'martin@front-matter.io'} )
    fg.link( href='https://front-matter.io', rel='alternate' )
    fg.link( href=f'https://front-matter.io/{subject_area}/feed.xml', rel='self' )
    fg.language('en')

    for index, row in subsets[subject_area].iterrows():
        fe = fg.add_entry(order='append')
        fe.title(row['title'] + f" ({row['tweets']} tweets)")
        fe.author(ast.literal_eval(row['authors']))
        fe.id('https://doi.org/' + row['doi'])
        fe.link({'href': 'https://doi.org/' + row['doi'], 'title': row['title'] + f" ({row['tweets']} tweets)"})
        fe.description(row['abstract'])
        fe.category({ 'term': row['subject-area']})
        fe.published(datetime.fromisoformat(row['posted'] + 'T00:00:00+00:00'))
                 
    fg.atom_file(f'{subject_area}/feed.xml')